# Lista 1 : Recuperação de informação 

### Imports... 

In [1]:
import numpy as np
import pandas as pd
import os

import nltk
from nltk.corpus import machado, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import PortugueseStemmer

import string

from collections import defaultdict

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
nltk.download('stopwords')
nltk.download('machado')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gabi_28_gabi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package machado to
[nltk_data]     /Users/gabi_28_gabi/nltk_data...
[nltk_data]   Package machado is already up-to-date!


True

In [3]:
textos = []
for i in machado.fileids():
    textos.append(machado.raw(i))

swu = stopwords.words('portuguese') + list (string.punctuation)
stemmer = PortugueseStemmer()    

In [4]:
# Sem stemming
textos_normal = []
for texto in textos:
    tlimpo = [token.lower() for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    textos_normal.append(tlimpo)

In [5]:
# Com stemming
textos_stem = []
for texto in textos_normal:
    tlimpo = [stemmer.stem(token) for token in texto]
    textos_stem.append(tlimpo)


In [6]:
print(textos_normal[0][0:10])
print(textos_stem[0][0:10])

['conto', 'contos', 'fluminenses', '1870', 'contos', 'fluminenses', 'texto', 'fonte', 'obra', 'completa']
['cont', 'cont', 'fluminens', '1870', 'cont', 'fluminens', 'text', 'font', 'obra', 'complet']


In [7]:
indice_normal = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_normal):
    for term in t:
        indice_normal[term].add(tid)

In [8]:
indice_stem = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_stem):
    for term in t:
        indice_stem[term].add(tid)


In [9]:
def busca(consulta, indice):
    
    tokens = WordPunctTokenizer().tokenize(consulta)

    resultado = set()
    operador = 'or'
    
    for token in tokens:
        if token == 'and':
            operador = 'and'
            
        elif token == 'or':
            operador = 'or'
        
        elif token == 'not':
            operador = 'not'
            
        else:
            resultado_temp = indice[token]
            if operador == 'and':
                resultado = resultado & resultado_temp
            
            elif operador == 'not':
                resultado = resultado - resultado_temp
                
            else:
                resultado = resultado | resultado_temp
    
    return resultado

## Exercício 1 

##### Baseando-se no indice invertido construído na prática 1, Calcule a diferença de revocação com e sem a utilização de "stemming", ou truncagem na construção do índice.

In [10]:
def PreRev(recuperados, relevantes):
    
    # precisao 
    pre = len(recuperados &  relevantes)/len(recuperados)
    # revocacao
    rev = len(recuperados &  relevantes)/len(relevantes)
    
    return pre,rev

In [11]:
# Palavra a ser consultada
palavra = 'contos'

In [12]:
# Resultados da busca
res_normal = busca(palavra,indice_normal)
res_stem = busca(stemmer.stem(palavra), indice_stem)

print('Quantidade de Resultados Stemmizado: ' + str(len(res_stem)))

print('Quantidade de Resultados Não Stemmizado: ' + str(len(res_normal)))

Quantidade de Resultados Stemmizado: 219
Quantidade de Resultados Não Stemmizado: 74


In [13]:
print('Precisão: ' +  str(PreRev(res_normal, res_stem)[0]))
print('Revocação: ' + str(PreRev(res_normal, res_stem)[1]))

Precisão: 1.0
Revocação: 0.3378995433789954


## Exercício 2 

#### Crie grupos de equivalência para alguns termos de busca e calcule a diferença em termos de revocação e, possivelmente precisão, na resposta a consultas expandidas e não expandidas. Dica: use tempos verbais, pluralização, sinônimos, etc.

In [14]:
termos = ['contos', 'conto', 'contar', 'contando', 'narração', 'narrações'] 
#termos = ['ator', 'atores','atriz','atrizes']

In [15]:
res_normal2 = set()

print('Documentos encontrados sem Stemming:')
for termo in termos:
    res = busca(termo, indice_normal)
    res_normal2.update(res)
    print(' {}: {}'.format(termo, len(res)))
print('\n')
print('TOTAL: {}'.format(len(res_normal2)))

Documentos encontrados sem Stemming:
 contos: 74
 conto: 166
 contar: 128
 contando: 58
 narração: 84
 narrações: 13


TOTAL: 201


In [16]:
res_stem2 = set()

print('Documentos encontrados sem Stemming:')
for termo in termos:
    res = busca(stemmer.stem(termo), indice_stem)
    res_stem2.update(res)
    print(' {}: {}'.format(termo, len(res)))
print('\n')
print('TOTAL: {}'.format(len(res_stem2)))

Documentos encontrados sem Stemming:
 contos: 219
 conto: 219
 contar: 219
 contando: 219
 narração: 84
 narrações: 13


TOTAL: 221


In [17]:
print('Precisão: ' +  str(PreRev(res_normal2, res_stem2)[0]))
print('Revocação: ' + str(PreRev(res_normal2, res_stem2)[1]))


Precisão: 1.0
Revocação: 0.9095022624434389


## Exercício 3 

#### Implemente uma expansão de consulta por meio da correção ortográfica. Utilize o corretor ortográfico Pyenchant para fazer as correções.

In [18]:
import enchant 

In [ ]:
# dicionario
d = enchant.Dict()

In [28]:
def corrige_palavra(palavra):
    
    if not d.check(palavra):
        
        print('Você quis dizer: ')
        for p in d.suggest(palavra)[:5]:
            print('-', p)


In [ ]:
corrige_palavra('ontos')

In [22]:
def busca_correta(palavra,ind,dic):

    # faz a verificação ortográfica e a busca sobre os termos possíveis
    
    if not dic.check(palavra):
        resultado = set()
        for token in dic.suggest(palavra):
            resultado = resultado | ind[token]
    else:
        resultado = ind[termo]
        
    return resultado

def busca2(consulta, indice):
    
    d = enchant.Dict("pt_BR")
    tokens = WordPunctTokenizer().tokenize(consulta)

    resultado = set()
    operador = 'or'
    
    for token in tokens:
        if token == 'and':
            operador = 'and'
            
        elif token == 'or':
            operador = 'or'
        
        elif token == 'not':
            operador = 'not'
            
        else:
            
            #resultado_temp = indice[token]
            # Caso um termo esteja escrito errado, faz-se a busca sobre todas as correções sugeridas
            resultado_temp = busca_correta(token, indice, d)
            
            if operador == 'and':
                resultado = resultado & resultado_temp
            
            elif operador == 'not':
                resultado = resultado - resultado_temp
                
            else:
                resultado = resultado | resultado_temp
    
    return resultado

In [23]:
# Palavra a ser consultada
palavra = 'ontos'

In [24]:
# Resultados da busca usando a função criada para os exercícios anteriores (busca)
res_normal = busca(palavra,indice_normal)
res_stem = busca(stemmer.stem(palavra), indice_stem)

print('Quantidade de Resultados Stemmizado: ' + str(len(res_stem)))

print('Quantidade de Resultados Não Stemmizado: ' + str(len(res_normal)))

Quantidade de Resultados Stemmizado: 0
Quantidade de Resultados Não Stemmizado: 0


In [ ]:
# Resultados da busca usando a função adaptada que usa a correção ortográfica (busca2)
res_normal = busca2(palavra,indice_normal)
res_stem = busca2(stemmer.stem(palavra), indice_stem)

print('Quantidade de Resultados Stemmizado: ' + str(len(res_stem)))

print('Quantidade de Resultados Não Stemmizado: ' + str(len(res_normal)))

## Exercício 4 

#### Implemente um indice invertido que permita consulta por frases, conforme definido na aula 2.

In [30]:
textos_limpos = []
for texto in textos:
    tlimpo = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    textos_limpos.append(tlimpo)
    
indice_frase = defaultdict(lambda: defaultdict(list))

for texto_id, texto in enumerate(textos_limpos):
    for token_id, token in enumerate(texto):
        indice_frase[token][texto_id].append(token_id)


In [31]:
indice_frase[stemmer.stem("Capitu")].keys()

dict_keys([230])

In [32]:
len(indice_frase[stemmer.stem("Capitu")][230])

341

In [33]:
def proximidade_termos(lista1, lista2, distancia):
    resultado = []
    
    for i in lista1:
        for j in lista2:
            if abs(i-j) <= distancia:
                resultado.append(round((i+j)/2))
                
    return resultado

In [34]:
def busca_frase(frase, indice, distancia=1):
    
    tokens = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(frase) if token not in swu]
    
    # Verificar quais documentos possuem todos os termos
    matches = indice[tokens[0]].keys()
    
    for token in tokens[1:]:
        matches = matches & indice[token].keys()    
        
    # Comparar as posições dentro dos documentos em comum
    resultado = defaultdict(list)
    
    for documento in matches:
        
        resultado_temp = indice[tokens[0]][documento]
        
        for token in tokens[1:]:
            resultado_temp = proximidade_termos(resultado_temp,indice[token][documento],distancia)
            
        if resultado_temp:
            resultado[documento] = resultado_temp
            
    return resultado

In [35]:
def busca_frase(frase, indice):
    tokens = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(frase) if token not in swu]
 
    matches = defaultdict(lambda:defaultdict(lambda:set([])))
    for token in tokens:
        matches[token] = indice[token]
    
    doc_pos = set(matches[tokens[0]])
    for tok in tokens:
        doc_pos = doc_pos.intersection(set(matches[token]))
        
    resultado = set([]) 
    for doc in doc_pos:
        for pos in matches[tokens[0]][doc]:
            if all((pos + token_id + 1 in matches[token][doc]) for token_id, token in enumerate(tokens[1:len(tokens)])):
                resultado.add(doc) 
                
    return resultado

In [36]:
def busca_frase(frase, indice):
    tokens = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(frase) if token not in swu]
 
    matches = defaultdict(lambda:defaultdict(lambda:set([])))
    for token in tokens:
        matches[token] = indice[token]
    
    doc_pos = set(matches[tokens[0]])
    for tok in tokens:
        doc_pos = doc_pos.intersection(set(matches[token]))
        
    resultado = set([]) 
    for doc in doc_pos:
        for pos in matches[tokens[0]][doc]:
            if all((pos + token_id + 1 in matches[token][doc]) for token_id, token in enumerate(tokens[1:len(tokens)])):
                resultado.add(doc) 
                
    return resultado

In [37]:
busca_frase("olhos de cigana oblíqua e dissimulada", indice_frase)

{230}

## Exercício 5 

#### Modifique a solução acima para permitir respostas alternativas caso a frase não retorne resultados. Por exemplo, retornar, documentos que contenham parte da frase, ou uma busca booleana simples combinando as palavras da frase.

In [38]:
import whoosh
from whoosh.index import create_in, open_dir
from whoosh.fields import *
from whoosh import qparser
from whoosh.qparser import QueryParser

In [39]:
schema = Schema(content=TEXT(phrase=True, stored=True))

if os.path.exists('indexdir'):
    ix = open_dir('indexdir')
else:
    os.mkdir('indexdir')
    ix = create_in("indexdir", schema)
    writer = ix.writer()
    for txt in textos:
        writer.add_document(content=txt)
    writer.commit()


searcher = ix.searcher()

In [40]:
def busca_frase_alt(frase, indice):
        
    tokens = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(frase) if token not in swu]
 
    resultado = busca_frase(frase, indice)
                
    if resultado == set([]):
        searcher = ix.searcher()
        query = QueryParser("content", ix.schema).parse(frase)
        resultado_alt = searcher.search(query)
        return resultado_alt.docs()
    else:
        return resultado

In [41]:
busca_frase_alt("olhos de cigana dissimulada e oblíqua", indice_frase)

{230}

In [42]:
busca_frase_alt("ressaca de olhos", indice_frase)

{82, 193, 224, 230, 244}